In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("../../")
sys.path.append("../../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import SUnet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
net1_W = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_64.npy")
net2_W = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_128.npy")
net3_W = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_200.npy")
net4_W = np.load("/home/vib9/src/SL-Net/superlayer/models/superblocks/SLN_256.npy")

In [3]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [4]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [5]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')

In [6]:
enc_nf = [32, 32, 32, 32]
dec_nf = [32, 32, 32, 32]
net1 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False, W=net1_W)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

(32, 64, 3, 3)


INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.37batch/s]
                                                                    INFO: Validation Dice Loss: 0.1095788871030231
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 21.75batch/s]
                                                                    INFO: Validation Dice Loss: 0.08875274453516845
Validation round: 100%|██████████| 91/91 [00:05<00:00, 21.85batch/s]
                                                                    INFO: Validation Dice Loss: 0.07914412799444827
Validation round:  98%|█████████▊| 89/

Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.07254597135297544
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.15batch/s]
                                                                    INFO: Validation Dice Loss: 0.0712386660359718
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.07056529304155937
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 22.99batch/s]
                                                                    INFO: Validation Dice Loss: 0.0695539469738583
Epoch 1/8: 100%|██████████| 5865/5865 [01:13<00:00, 79.97img/s, loss (batch)=0.223] 
INFO: Checkpoint 1 saved !
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.54batch/s]
                                                                    INFO: Validat

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.06963537854480219
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.69batch/s]
                                                                    INFO: Validation Dice Loss: 0.07086667647728553
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.06916676101448772
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 20.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.06881858834198543
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.06747156643605494
Epoch 2/8:  68%|██████▊   | 3968/5865 [00:56<00:14, 130.47img/s, loss (batc

Validation round:  93%|█████████▎| 85/91 [00:04<00:00, 17.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.06795417181738131
Validation round: 100%|██████████| 91/91 [00:05<00:00, 23.11batch/s]
                                                                    INFO: Validation Dice Loss: 0.06874856180869617
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.96batch/s]
                                                                    INFO: Validation Dice Loss: 0.06877421514018552
Epoch 2/8: 100%|██████████| 5865/5865 [01:23<00:00, 70.13img/s, loss (batch)=0.222] 
INFO: Checkpoint 2 saved !
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.79batch/s]
                                                                    INFO: Validation Dice Loss: 0.0681554709489529
Validation round:  52%|█████▏    | 47/91 [00:03<00:02, 16.39batch/s]


...

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.04batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816248980524776
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.66batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816204940224742
Validation round: 100%|██████████| 91/91 [00:04<00:00, 21.47batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816192135051057
Epoch 6/8: 100%|██████████| 5865/5865 [01:24<00:00, 69.50img/s, loss (batch)=0.233] 
INFO: Checkpoint 6 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.01batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816233162369047
Validation round: 100%|██████████| 91/91 [00:04<00:00, 23.34batch/s]
                                                                    INFO: Valid

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.08batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816157543069713
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.69batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816252452004087
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.48batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816207191773824
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 20.57batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816277440105166
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.11batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816286176115602
Epoch 7/8:  81%|████████▏ | 4776/5865 [01:06<00:08, 135.12img/s, loss (batc

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.23batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816250175892652
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816150739297762
Epoch 7/8: 100%|██████████| 5865/5865 [01:23<00:00, 70.22img/s, loss (batch)=0.276] 
INFO: Checkpoint 7 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.41batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816209828133112
Validation round:  97%|█████████▋| 88/91 [00:04<00:00, 19.46batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816154914897876
Validation round:  96%|█████████▌| 87/91 [00:04<00:00, 18.67batch/s]


Validation round: 100%|██████████| 91/91 [00:05<00:00, 22.10batch/s]
                                                                    INFO: Validation Dice Loss: 0.0681610691187146
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.29batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816075512996086
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.10batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816116433877212
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.58batch/s]
                                                                    INFO: Validation Dice Loss: 0.0681613408602201
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 18.52batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816068275289221
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.91batch/s]
        

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.37batch/s]
                                                                    INFO: Validation Dice Loss: 0.06816262358820045
Epoch 8/8: 100%|██████████| 5865/5865 [01:25<00:00, 68.98img/s, loss (batch)=0.257] 
INFO: Checkpoint 8 saved !


In [7]:
enc_nf = [64, 64, 64, 64]
dec_nf = [64, 64, 64, 64]
net2 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False, W=net2_W)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Epoch 1/8:   0%|          | 0/5865 [00:00<?, ?img/s]

(64, 128, 3, 3)


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.53batch/s]
                                                                    INFO: Validation Dice Loss: 0.15026271719853956
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.42batch/s]
                                                                    INFO: Validation Dice Loss: 0.13284751207946421
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.13batch/s]
                                                                    INFO: Validation Dice Loss: 0.12466157088567922
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.56batch/s]
                                                                    INFO: Validation Dice Loss: 0.12201736638179192
Validation round:  54%|█████▍    | 49/91 [00:03<00:03, 13.79batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.86batch/s]
                                                                    INFO: Validation Dice Loss: 0.10685005153600986
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.22batch/s]
                                                                    INFO: Validation Dice Loss: 0.11096282303333282
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.64batch/s]
                                                                    INFO: Validation Dice Loss: 0.10365006450440857
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.12batch/s]
                                                                    INFO: Validation Dice Loss: 0.10566790704871272
Epoch 1/8: 100%|██████████| 5865/5865 [01:22<00:00, 71.38img/s, loss (batch)=0.351] 
INFO: Checkpoint 1 saved !
Validation round:  48%|████▊     | 44/91 [00:03<00:04, 10.44batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.19batch/s]
                                                                    INFO: Validation Dice Loss: 0.1011298031924845
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 15.97batch/s]
                                                                    INFO: Validation Dice Loss: 0.09921185123723941
Validation round: 100%|██████████| 91/91 [00:05<00:00, 22.07batch/s]
                                                                    INFO: Validation Dice Loss: 0.10473613981362227
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 21.93batch/s]
                                                                    INFO: Validation Dice Loss: 0.0974466568166083
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.95batch/s]


...

Validation round: 100%|██████████| 91/91 [00:05<00:00, 18.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.0900678561775239
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.04batch/s]
                                                                    INFO: Validation Dice Loss: 0.09006884030915879
Epoch 6/8: 100%|██████████| 5865/5865 [01:30<00:00, 64.62img/s, loss (batch)=0.347] 
INFO: Checkpoint 6 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.88batch/s]
                                                                    INFO: Validation Dice Loss: 0.09006981477960126
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.50batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007053863216233
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.58batch/s]
                                                                    INFO: Valida

Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.83batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007238686739744
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.54batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007165638300088
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.80batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007162879129033
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.67batch/s]
                                                                    INFO: Validation Dice Loss: 0.0900723376608157
Validation round:  63%|██████▎   | 57/91 [00:03<00:02, 15.51batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.05batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007362587438834
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.83batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007359377957963
Epoch 7/8: 100%|██████████| 5865/5865 [01:29<00:00, 65.50img/s, loss (batch)=0.355] 
INFO: Checkpoint 7 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 16.20batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007397244919788
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 22.01batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007338278896206
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.09batch/s]
                                                                    INFO: Valid

Validation round:  97%|█████████▋| 88/91 [00:05<00:00, 18.07batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007420407219248
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.34batch/s]
                                                                    INFO: Validation Dice Loss: 0.0900755374804958
Validation round:  97%|█████████▋| 88/91 [00:05<00:00, 20.45batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007640682406477
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 22.17batch/s]
                                                                    INFO: Validation Dice Loss: 0.0900774666077488
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.65batch/s]


                                                                    INFO: Validation Dice Loss: 0.09007746415151345
Validation round:  95%|█████████▍| 86/91 [00:05<00:00, 17.11batch/s]
                                                                    INFO: Validation Dice Loss: 0.09007704454463916
Epoch 8/8: 100%|██████████| 5865/5865 [01:29<00:00, 65.85img/s, loss (batch)=0.355] 
INFO: Checkpoint 8 saved !


In [ ]:
enc_nf = [100, 100, 100, 100]
dec_nf = [100, 100, 100, 100]
net3 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False, W=net3_W)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net3.to(device=device)
train_scores3, val_scores3, train_var_3, val_var_3 = train_net(net=net3,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 5865 examples
INFO: Creating dataset with 732 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   5865
        Validation size: 732
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Epoch 1/8:   0%|          | 0/5865 [00:00<?, ?img/s]

(100, 200, 3, 3)


Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 17.38batch/s]
                                                                    INFO: Validation Dice Loss: 0.09138654426231489
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.66batch/s]
                                                                    INFO: Validation Dice Loss: 0.09764578119739072
Validation round:  98%|█████████▊| 89/91 [00:04<00:00, 20.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.09021581635697858
Validation round:  12%|█▏        | 11/91 [00:01<00:17,  4.47batch/s]


Validation round: 100%|██████████| 91/91 [00:05<00:00, 21.55batch/s]
                                                                    INFO: Validation Dice Loss: 0.09417994271267902
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.27batch/s]
                                                                    INFO: Validation Dice Loss: 0.0930941077870327
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 20.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.08565180110079902
Validation round:  46%|████▌     | 42/91 [00:02<00:02, 18.52batch/s]


Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 20.76batch/s]
                                                                    INFO: Validation Dice Loss: 0.09089948605377596
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.30batch/s]
                                                                    INFO: Validation Dice Loss: 0.08876106527807949
Epoch 1/8: 100%|██████████| 5865/5865 [01:54<00:00, 51.29img/s, loss (batch)=0.387] 
INFO: Checkpoint 1 saved !
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 20.71batch/s]
                                                                    INFO: Validation Dice Loss: 0.0885021094601233
Validation round:  71%|███████▏  | 65/91 [00:04<00:01, 19.03batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.12batch/s]
                                                                    INFO: Validation Dice Loss: 0.08480090746185281
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.46batch/s]
                                                                    INFO: Validation Dice Loss: 0.08598748373461294
Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.17batch/s]
                                                                    INFO: Validation Dice Loss: 0.08524067839095881
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 17.10batch/s]
                                                                    INFO: Validation Dice Loss: 0.08548715473203869
Validation round:  14%|█▍        | 13/91 [00:01<00:19,  4.09batch/s]


Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.08482236865457597
Validation round: 100%|██████████| 91/91 [00:05<00:00, 24.13batch/s]
                                                                    INFO: Validation Dice Loss: 0.08535964050135769
Validation round: 100%|██████████| 91/91 [00:05<00:00, 19.79batch/s]
                                                                    INFO: Validation Dice Loss: 0.08488650808295051
Validation round:  51%|█████     | 46/91 [00:03<00:02, 17.48batch/s]


Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.65batch/s]
                                                                    INFO: Validation Dice Loss: 0.08508173793881804
Epoch 2/8: 100%|██████████| 5865/5865 [02:10<00:00, 44.83img/s, loss (batch)=0.384]
INFO: Checkpoint 2 saved !
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.14batch/s]
                                                                    INFO: Validation Dice Loss: 0.08502955177983085
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.44batch/s]
                                                                    INFO: Validation Dice Loss: 0.08523487692678368
Validation round:  75%|███████▍  | 68/91 [00:04<00:01, 16.51batch/s]


Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 18.84batch/s]
                                                                    INFO: Validation Dice Loss: 0.085218703353798
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 21.19batch/s]
                                                                    INFO: Validation Dice Loss: 0.08518249607020682
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 21.01batch/s]
                                                                    INFO: Validation Dice Loss: 0.08517156222036906
Validation round:  98%|█████████▊| 89/91 [00:05<00:00, 19.81batch/s]
                                                                    INFO: Validation Dice Loss: 0.08516861064420951
Validation round:   0%|          | 0/91 [00:00<?, ?batch/s]


Validation round: 100%|██████████| 91/91 [00:05<00:00, 20.35batch/s]
                                                                    INFO: Validation Dice Loss: 0.08516703013862882
Validation round:  99%|█████████▉| 90/91 [00:05<00:00, 19.78batch/s]
                                                                    INFO: Validation Dice Loss: 0.08516402247842851
Epoch 3/8:  89%|████████▉ | 5232/5865 [01:56<00:08, 70.45img/s, loss (batch)=0.39] 

In [ ]:
enc_nf = [128, 128, 128, 128]
dec_nf = [128, 128, 128, 128]
net4 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False, W=net4_W)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net4.to(device=device)
train_scores4, val_scores4, train_var_4, val_var_4 = train_net(net=net4,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

In [ ]:
print("configuring combined plots")
domain = len(train_scores1)
x_values = [i+1 for i in range(domain)]
             
a1 = plt.subplot(1,2,1)
a1.set_ylim([0, 0.5])
plt.title("Complex SLN 64 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores1, train_var_1)]
ziptraindown = [a - b for a, b in zip(train_scores1, train_var_1)]
zipvalup = [a + b for a, b in zip(val_scores1, val_var_1)]
zipvaldown = [a - b for a, b in zip(val_scores1, val_var_1)]

plt.plot(x_values, train_scores1, color="blue", label="train")
a1.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores1, color="orange", label="val")
a1.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a2 = plt.subplot(1,2,2)
a2.set_ylim([0, 0.5])
plt.title("Complex SLN 128 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores2, train_var_2)]
ziptraindown = [a - b for a, b in zip(train_scores2, train_var_2)]
zipvalup = [a + b for a, b in zip(val_scores2, val_var_2)]
zipvaldown = [a - b for a, b in zip(val_scores2, val_var_2)]

plt.plot(x_values, train_scores2, color="blue", label="train")
a2.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores2, color="orange", label="val")
a2.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

a3 = plt.subplot(1,2,1)
a3.set_ylim([0, 0.5])
plt.title("Complex SLN 200 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores3, train_var_3)]
ziptraindown = [a - b for a, b in zip(train_scores3, train_var_3)]
zipvalup = [a + b for a, b in zip(val_scores3, val_var_3)]
zipvaldown = [a - b for a, b in zip(val_scores3, val_var_3)]

plt.plot(x_values, train_scores3, color="blue", label="train")
a3.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores3, color="orange", label="val")
a3.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

a4 = plt.subplot(1,2,2)
a4.set_ylim([0, 0.5])
plt.title("Complex SLN 256 Superblock")
plt.xlabel("Mini-epochs")
plt.ylabel("Dice Loss")

ziptrainup = [a + b for a, b in zip(train_scores4, train_var_4)]
ziptraindown = [a - b for a, b in zip(train_scores4, train_var_4)]
zipvalup = [a + b for a, b in zip(val_scores4, val_var_4)]
zipvaldown = [a - b for a, b in zip(val_scores4, val_var_4)]

plt.plot(x_values, train_scores4, color="blue", label="train")
a4.fill_between(x_values, ziptrainup, ziptraindown, facecolor='lightskyblue', alpha=0.5)
plt.plot(x_values, val_scores4, color="orange", label="val")
a4.fill_between(x_values, zipvalup, zipvaldown, facecolor='navajowhite', alpha=0.5)

plt.legend()
plt.grid()

plt.show()

plt.close()